# The Jamming DB

This NB assumes `REPOROOT/data/testing/jamming.db` already exists.
To create the DB, from REPOROOT, run 
```python scripts/initialize_db.py data/testing/jamming.db```

Present notebook tests insert/delete/update operations, so you'll need to rerun the build.

In [ ]:
! cd ../../; python scripts/initialize_db.py data/testing/jamming.db --force_rebuild

In [ ]:
import sys
from pathlib import Path

REPO_ROOT = Path("./").absolute().parent.parent
sys.path.append(str(REPO_ROOT))
from jamdb.db import BackendSQLite, DBError #, UniqueConstraintError, FKConstraintError

DB_FILE = REPO_ROOT / "data/testing/jamming.db"

db_handler = BackendSQLite(DB_FILE)

In [ ]:
## Tests to run on the test db.

# db_handler.query("SELECT * FROM Key")["mode_id"].value_counts()

# try:
#     db_handler.delete_row("Person", "paul_k")
# except DBError:
#     pass # we expect FK error

# db_handler.delete_row("Song", "work_song")

# db_handler.insert_row("PersonInstrument", {"id": "made_up", "person_id": "paul_k", "instrument_id": "trumpet"})

# try:
#     db_handler.insert_row("PersonInstrument", {"id": "made_up_2", "person_id": "paul_k", "instrument_id": "gong"})
# except DBError:
#     pass # we expect FK Error

# try:
#     db_handler.update_row("PersonInstrument", "made_up", {"id": "made_up", "instrument_id": "a_sax"})
# except DBError:
#     pass # expect error since trying to update pk

# try:
#     db_handler.update_row("PersonInstrument", "made_up", {"instrument_id": "e_guitar"})
# except DBError:
#     pass # expect Uniqueness error

# try:
#     db_handler.update_row("PersonInstrument", "made_up", {"instrument_id": "gong"})
# except DBError:
#     pass # expect FK error

# db_handler.update_row("PersonInstrument", "made_up", {"instrument_id": "t_sax"})

# db_handler.get_row("PersonInstrument", "made_up")

# db_handler.rename_primary_key("PersonInstrument", "made_up", "paul_k:t_sax")

# db_handler.get_row("PersonInstrument", "paul_k:t_sax")

# db_handler.get_row("PersonInstrument", "paul_k:e_guitar")

# db_handler.rename_primary_key("PersonInstrument", "paul_k:e_guitar", "paul_k:electric_geetar")

# db_handler.get_row("PersonInstrument", "paul_k:electric_geetar")

# db_handler.get_row("PersonInstrument", "travis_p:a_sax")

# try:
#     db_handler.delete_row("PersonInstrument", "travis_p:a_sax")
# except DBError:
#     pass # expect FK error

# db_handler.rename_primary_key("Person", "mike_p", "mikey_p")

# db_handler.rename_primary_key("Key", "A_minor", "MY NER")

# db_handler.rename_primary_key("Mode", "minor", "maynor")

In [3]:
import sys
from pathlib import Path

REPO_ROOT = Path("./").absolute().parent.parent
sys.path.append(str(REPO_ROOT))
from jamdb.db import BackendSQLite #, DBError, UniqueConstraintError, FKConstraintError

DB_FILE = REPO_ROOT / "data/jamming.db"

db_handler = BackendSQLite(DB_FILE)

In [13]:
table_name, pk = "Person", "travis_p"
got = db_handler.get_row(table_name, pk)
got

{'id': 'travis_p',
 'public_name': 'Travis P',
 'full_name': 'Travis to the P Perry',
 'facebook': '',
 'instagram': '',
 'youtube': '',
 'other_contact': ''}

In [14]:
import copy
db_handler.update_row(table_name, pk, {"full_name": "Travis Perry"})

In [8]:
help(db_handler.update_row)

Help on method update_row in module jamdb.db:

update_row(table_name, primary_key, row, commit=True) method of jamdb.db.BackendSQLite instance



In [ ]:
# import sys
# from pathlib import Path

# REPO_ROOT = Path("./").absolute().parent.parent
# sys.path.append(str(REPO_ROOT))
# from jamdb.db import BackendSQLite #, DBError, UniqueConstraintError, FKConstraintError

# DB_FILE = REPO_ROOT / "data/jamming.db"

# db_handler = BackendSQLite(DB_FILE)

In [18]:
# db_handler.get_row("Person", "t_the_p")
# db_handler.delete_row("Person", "t_the_p")

db_handler.table_names()

['Composer',
 'EventGen',
 'EventOcc',
 'Genre',
 'Instrument',
 'Key',
 'Mode',
 'Person',
 'PersonInstrument',
 'Setlist',
 'SetlistSongs',
 'Song',
 'SongLearn',
 'SongPerform',
 'SubGenre',
 'Venue']

In [29]:
db_handler.entities.get("Person").primary_key

'id'

In [ ]:
db_handler._create_erd()

In [ ]:
# row = db_handler._get_relations().iloc[17]
row = {'left_table': 'SongPerform',
 'left_key': 'instrument_id',
 'right_table': 'Instrument',
 'right_key': 'id'}


def groupby_list(df, groupby_col, value_col):
    df_g = df.copy()
    df_g[value_col] = df_g[value_col].apply(lambda x: [x])
    return df_g.groupby(groupby_col)[value_col].sum().to_dict()

def get_key_mapping_for_fk_relation(self, row):
    referer_name = row['left_table']
    fk_referer = row['left_key']
    refered_name = row['right_table']
    fk_refered = row['right_key']
    pk_referer = self.entities[referer_name].primary_key
    pk_refered = self.entities[refered_name].primary_key
    
    query = f"""
        SELECT
            c.{pk_referer} as pk_referer,
            c.{fk_referer} as fk_referer,
            m.{pk_refered} as pk_refered,
            m.{fk_refered} as fk_refered
        FROM
            {referer_name} as c
        JOIN
            {refered_name} as m
        ON c.{fk_referer} = m.{fk_refered}
    """
    result = self.query(query)
    output = [
        {
            "referer": referer_name,
            "refered": refered_name,
            "referer_pk": pk_referer,
            "referer_fk": fk_referer,
            "refered_pk": pk_refered,
            "refered_fk": fk_refered,
            "referer_to_refered": groupby_list(result, "pk_referer", "pk_refered"),
            "refered_to_referer": groupby_list(result, "pk_refered", "pk_referer"),            
        }
    ]
    
    # output = [
    #     {
    #         "referer": referer_name,
    #         "refered": refered_name,
    #         "referer_pk": pk_referer,
    #         "referer_fk": fk_referer,
    #         "refered_pk": pk_refered,
    #         "refered_fk": fk_refered,
    #         "referer_to_refered": groupby_list(result, "pk_referer", "pk_refered"),
    #     },
    #     {
    #         "referer": refered_name,
    #         "refered": referer_name,
    #         "referer_pk": pk_refered,
    #         "referer_fk": fk_refered,
    #         "refered_pk": pk_referer,
    #         "refered_fk": fk_referer,
    #         "referer_to_refered": groupby_list(result, "pk_refered", "pk_referer"),
    #     }
    # ]
    return output

key_mappings = []
for _, row in db_handler._get_relations().iterrows():
    key_mappings.extend(get_key_mapping_for_fk_relation(db_handler, row))


In [ ]:
key_mappings

In [ ]:
pk_referer = db_handler.entities[referer_name].primary_key
pk_refered = db_handler.entities[refered_name].primary_key
referer_cols = [f"c.{pk_referer} as referer_pk", f"c.{fk_referer} as referer_fk"]
referer_table = db_handler.query(f"SELECT {','.join(referer_cols)} FROM {referer_name} as c")
referer_table

In [ ]:
def groupby_list(df, groupby_col, value_col):
    df_g = df.copy()
    df_g[value_col] = df_g[value_col].apply(lambda x: [x])
    return df_g.groupby(groupby_col)[value_col].sum().to_dict()

query = f"""
SELECT
    c.{pk_referer} as pk_referer,
    c.{fk_referer} as fk_referer,
    m.{pk_refered} as pk_refered,
    m.{fk_refered} as fk_refered
FROM
    {referer_name} as c
JOIN
    {refered_name} as m
ON c.{fk_referer} = m.{fk_refered}
"""
A = db_handler.query(query)

# groupby_list(A, "pk_referer", "pk_refered")

groupby_list(A, "pk_refered", "pk_referer")

In [ ]:
conn.get_row(table_name="Key", primary_key="D_minor")

In [ ]:
! echo $PYTHONPATH

In [ ]:
import graphviz
graphviz.__version__

In [ ]:
import sqlite3
sqlite3.__version__

In [ ]:
from jamdb.entities import Field

In [ ]:
# It is proving non-trivial to fetch various constraints from the db
# and some of if effectively comes back to parsing the `jamming.sql`
# Alternative, we could DEFINE all this stuff in a json, and then 
# create the `jamming.sql` and the DB itself from the json.

stuff = conn.query("select sql from sqlite_master where type='table' and name='EventGen'").iloc[0]["sql"]

# pattern = "unique"
# pattern = "not null"
pattern = "default"

entries = [
    tmp.strip() for tmp in stuff.splitlines()
    if pattern in tmp.lower()
]
entries

In [ ]:
relations = conn._get_relations()
entity = conn.entities["EventOcc"]
fields = {}

for column in entity.columns:
    # How to extract UNIQUE, REQUIRED, and DEFAULTs from sqllite metadata?
    unique = False
    required = False
    default = None
    
    relation = relations.query(f"left_table == '{entity.table_name}'").query(f"left_key == '{column}'")
    if len(relation) == 1:
        allowed_values = relation.iloc[0]["allowed_values"]
    else:
        allowed_values = None

    current_values = list(conn.query(f"SELECT DISTINCT {column} FROM {entity.table_name}")[column])

    field = Field(
        table_name=entity.table_name,
        field_name=column,
        required=required, 
        unique=unique,
        allowed_values=allowed_values,
        default=default,
        current_values=current_values
    )
    fields[column] = field

fields["event_gen_id"].allowed_values

In [ ]:
conn.query("SELECT * FROM _schema_columns")

In [ ]:
# table_names = conn._sorted_tables()
# idx = -1

idx +=1
table_name = table_names[idx]

print(table_name)
conn.query(f"SELECT * FROM {table_name}").head()

In [ ]:
# conn.query("SELECT * FROM PersonInstrument")
# conn.query("SELECT * FROM SongPerform")
# conn.query("SELECT * FROM EventGen")

In [ ]:
conn.conn.close()
conn._connect()

In [ ]:
conn.query("SELECT * FROM Key")

In [ ]:
conn.query("SELECT * FROM Song")["key_id"].drop_duplicates().sort_values()

In [ ]:
conn.query("SELECT * FROM SongPerform")["key_id"].drop_duplicates().sort_values()

In [ ]:
conn._connect()

In [ ]:
conn.get_row("PersonInstrument", "trav_p:alto_sax")

In [ ]:
conn.query("SELECT * FROM SongPerform")

In [ ]:
conn.query("SELECT * FROM Venue").head()

In [ ]:
print("\n".join(list(conn.entities.keys())))

In [ ]:
conn._create_erd()

In [ ]:
table = Table(conn, "PersonInstrument")

In [ ]:
conn.query(f"pragma foreign_key_list('{table_name}')")

In [ ]:
insert_person_inst({"id": "made_up", "person_id": "paul_k", "instrument_id": "gong"}, conn)

In [ ]:
table_name = "PersonInstrument"

stuff = conn.query(f"select sql from sqlite_master where type='table' and name='{table_name}'").iloc[0]["sql"]
print(stuff)

entries = [ tmp.strip() for tmp in stuff.splitlines() if tmp.lower().find("constraint")>=0 or tmp.lower().find("unique")>=0 ]
entries

In [ ]:
conn.execute(f"pragma index_list('{table_name}')").fetchall()

In [ ]:
conn.execute(f".schema {table_name}").fetchall()

In [ ]:
relations = []
for table_name in conn.table_names():
    relations.extend(
        {
            "left_table": table_name,
            "left_key": row["from"],
            "right_table": row["table"],
            "right_key": row["to"]
        }
        for _, row in conn.query(f"pragma foreign_key_list('{table_name}')").iterrows()
    )

for constraint in relations:
    constraint["allowed_values"] = list(
        conn.query(f'SELECT {constraint["right_key"]} FROM {constraint["right_table"]}')[constraint["right_key"]]
    )
relations

# for constraint in fks:
#     col_name = constraint["left_key"]
#     if constraint["left_key"] in row:
#         value = row[col_name]
        
#         constraint["allowed_values"] = list(
#             self.backend.query(f'SELECT {constraint["right_key"]} FROM {constraint["right_table"]}')[constraint["right_key"]]
#         )


# fks = (
#                 self.backend._get_relations().query(f'left_table == "{self.table_name}"')
#             ).to_dict(orient="records")
    
#             for constraint in fks:
#                 col_name = constraint["left_key"]
#                 if constraint["left_key"] in row:
#                     value = row[col_name]
                    
#                     constraint["allowed_values"] = list(
#                         self.backend.query(f'SELECT {constraint["right_key"]} FROM {constraint["right_table"]}')[constraint["right_key"]]
#                     )

In [ ]:
df = conn.query("SELECT table_name, column from _schema_columns")
df["column"] = df["column"].apply(lambda x: [x])
df.groupby("table_name")["column"].sum().to_dict()